# Computing for Economists: Data Wrangling





In this module, we will have you work through the following datasets to analyze social connectedness in Europe. This module is inspired by [Bailey et al. (2020)](https://pages.stern.nyu.edu/~jstroebe/PDF/BJRKSS_EuroSCI.pdf).

In [4]:
### Import Modules
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

In [3]:
!pip install geopandas

  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/c4/64/7d344cfcef5efddf9cf32f59af7f855828e9d74b5f862eddf5bfd9f25323/geopandas-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyogrio>=0.7.2 from https://files.pythonhosted.org/packages/21/c2/f4ce7e004550a1ff74c83465fb9d11507f2d77dfab54203320e68fcd3c90/pyogrio-0.9.0-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/30/bd/b9bd3761f08754e8dbb34c5a647db2099b348ab5da338e90980caf280e37/pyproj-3.6.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for shapely>=2.0.0 from https://files.pythonhosted.org/packages/2a/fb/e3f72b10a90e26bb1a92a38b3f30f3074ebac6d532f87848ac09c3e8a73b/shapely-2.0.4-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/

## Tidying the Data

1. The `data` folder contains data from the following sources. Read each of the data files into Python as Pandas DataFrames.
  - https://ec.europa.eu/eurostat/en/web/products-datasets/-/EDAT_LFSE_04 
    - This data contains the percent of population by educational attainment level, sex, and NUTS 2 regions in the European Union.
  - https://ec.europa.eu/eurostat/web/products-datasets/-/nama_10r_2hhinc
    - The data contains the income of households by NUTS 2 region.  
  - https://ec.europa.eu/eurostat/web/products-datasets/product?code=demo_r_pjanaggr3
    - Population by age, group, sex, and NUTS 2 region.
  - https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts
    - NUTS region shapefiles (use the geopandas library to read it as a GeoDataFrame).
  - https://media.githubusercontent.com/media/social-connectedness-index/euro_sci/master/_intermediate_data/geo_distance_dat.csv
    - Distance between pairs of NUTS 2 regions constructed by [Bailey et al. (2020)](https://pages.stern.nyu.edu/~jstroebe/PDF/BJRKSS_EuroSCI.pdf).
  - `gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv` from https://data.humdata.org/dataset/social-connectedness-index
    - This file contains the social connectedness index between NUTS 2 regions and all Global Administrative Areas (GADM). We will only consider NUTS 2 to NUTS 2 connections. Social connectedness is a snapshot of Facebook links as of July 2019.
    - Note: this file is very large so we recommend reading the file directly into the notebook at the following link: https://data.humdata.org/dataset/e9988552-74e4-4ff4-943f-c782ac8bca87/resource/5de92e01-606e-4e4d-ad7c-4a3d493a0cc3/download/gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv


In [48]:
# read in data
pop_raw = pd.read_csv('data/estat_demo_r_pjanaggr3.tsv', sep='\t')
educ_raw = pd.read_csv('data/estat_edat_lfse_04.tsv', sep='[,\t]')
inc_raw = pd.read_csv('data/estat_nama_10r_2hhinc.tsv', sep='[,\t]')
soc_con = pd.read_csv('data/gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv', sep='\t')
# soc_con = pd.read_csv('https://data.humdata.org/dataset/e9988552-74e4-4ff4-943f-c782ac8bca87/resource/5de92e01-606e-4e4d-ad7c-4a3d493a0cc3/download/gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv', sep='\t')
dist = pd.read_csv('data/geo_distance_dat.csv')
geo = gpd.read_file('data/NUTS_RG_20M_2021_3035.shp/NUTS_RG_20M_2021_3035.shp')

/var/folders/g7/hdjg_nys3bl4y18x_kypq4rm0000gq/T/ipykernel_49592/95112532.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  educ_raw = pd.read_csv('data/estat_edat_lfse_04.tsv', sep='[,\t]')
/var/folders/g7/hdjg_nys3bl4y18x_kypq4rm0000gq/T/ipykernel_49592/95112532.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  inc_raw = pd.read_csv('data/estat_nama_10r_2hhinc.tsv', sep='[,\t]')


In [39]:
educ.columns

Index(['sex', 'isced11', 'age', 'geo\TIME_PERIOD', '2000 ', '2001 ', '2002 ',
       '2003 ', '2004 ', '2005 ', '2006 ', '2007 ', '2008 ', '2009 ', '2010 ',
       '2011 ', '2012 ', '2013 ', '2014 ', '2015 ', '2016 ', '2017 ', '2018 ',
       '2019 ', '2020 ', '2021 ', '2022 ', '2023'],
      dtype='object')


2. Clean the education data (`estat_edat_lfse_04.tsv`) and put it into "tidy" format. Check that there is a unique non-missing key for each row. The resulting data should have five columns and 299,088 rows.

In [44]:
# make education data tidy
educ = educ_raw.query("sex == 'T'")

educ = educ.drop(columns=['freq', 'unit', 'sex'])
educ = educ.rename(columns={'geo\TIME_PERIOD': 'geo'})


# rename variables

# reshape from wide to long

years = [str(year) for year in range(2000, 2024)]
educ = pd.melt(educ, id_vars=['isced11', 'age', 'geo'], value_name='pct_educ', var_name='year', ignore_index=False)

# pd.wide_to_long(dfw, stubnames=’price’, i=’product’, j=’market’, sep=’_’)
# pd.melt
educ

,isced11,age,geo,year,pct_educ
24924,ED0-2,Y20-24,AT,2000,14.9
24925,ED0-2,Y20-24,AT1,2000,15.3
24926,ED0-2,Y20-24,AT11,2000,: u
24927,ED0-2,Y20-24,AT12,2000,14.8
24928,ED0-2,Y20-24,AT13,2000,15.8
...,...,...,...,...,...
37381,ED5-8,Y30-34,UKM7,2023,:
37382,ED5-8,Y30-34,UKM8,2023,:
37383,ED5-8,Y30-34,UKM9,2023,:
37384,ED5-8,Y30-34,UKN,2023,:


3. Do the same for the income data (`estat_nama_10r_2hhinc.tsv`) and the population data (`estat_demo_r_pjanaggr3.tsv`).

In [57]:
inc = inc_raw

# go through a lot of effort to drop the freq variable
for col in inc.columns:
    if inc[col].nunique() == 1:
        inc = inc.drop(columns=[col,])

inc = inc.rename(columns={'geo\TIME_PERIOD': 'geo'})

4. Outer join the percent of the population age 25 to 64 in a NUTS 2 region receiving a tertiary education with the income per capita in purchasing power standard (PPS). Then outer join the education and income data with population data for the total population (for sex and age). Only include data at the NUTS 2 level.

  To obtain the income per capita in PPS, the national account should be the "balance of primary incomes/national income, net" and the direction of flow should be the "balance".

5. For each pair of NUTS 2 regions, obtain the absolute difference in the two variables constructed in problem 4 in 2019. Drop the French islands (starts with NUTS 2 region code "FRY").


6. Now we will merge the the 2019 data with the SCI data and the pairwise distance data. Create a dataframe with one row for each pair of NUTS 2 regions (as two separate columns) and the following variables:

*   Log scaled SCI
*   Log distance (add 1 to avoid zeros)
*   Percent tertiary education for User and Friend NUTS 2
*   Income per capita for User and Friend NUTS 2
*   Population for User and Friend NUTS 2
*   Absolute difference in tertiary education between User and Friend NUTS 2
*   Absolute difference in income per capita between User and Friend NUTS 2

  Note that the SCI data contains GADM areas, please only keep the NUTS 2 to NUTS 2 rows. Keeps all rows in the SCI data.


## Descriptive Statistics


1.   Each country is represented by the first two letters of the NUTS 2 index. Present a country-level population-weighted summary table for the percent people with tertiary education and income per capita in 2019. Include the mean, standard deviation, min, and max. Round all numeric outputs to two decimal places.

2. The SCI is scaled from 1 to 1,000,000 and measures the relative probability of a Facebook friendship link ([source](https://data.humdata.org/dataset/social-connectedness-index?)). If a number is two times higher, then a friendship link between two users from those NUTS 2 regions is twice as likely. Calculate  the mean, std, min, and max of log(scaled SCI) by country and merge it with the summary table from the previous part.

3. Compute the correlation between log(scaled SCI), log(distance), user region education and income, as well as differences in education and income between NUTS 2 regions. Is it higher or lower than what you expected?

## Data Visualization

1. We will ask you to plot two maps of social connections of (1) a country and (2) a NUTS 2 region to all other NUTS 2 regions. Remember the French islands should be dropped (these islands are far away from continental Europe and will therefore make the maps harder to visualize).

  * For the chosen country, aggregate the SCI to produce country to NUTS 2 region measures. Scaled SCI can be aggregated correctly using population shares following [SCI documentation](https://data.humdata.org/dataset/social-connectedness-index). For countries $i, j$ with regions $\{r_i\}, \{r_j\}$, aggregate SCI is given by:

  $$SCI_{i,j} = \sum_{\{r_i\}}\sum_{\{r_j\}} PopShare_{r_i} \times PopShare_{r_j} \times SCI_{r_i, r_j}$$

  where $PopShare_{r_i}$ is the share of country $i$'s population in region $r_i$.

  * Merge the aggregated SCI data with the GeoDataFrame (keep only NUTS 2 regions) for the applicable geographies.
  * Construct the bins for the map legend by first selecting a baseline SCI cutoff (e.g. 20th percentile of the **total data** that you are working with). Then calculate the subsequent cutoffs as $2\times$, $3\times$, $5\times$, $10\times$, $25\times$, $100\times, 1000\times$ that of the baseline cutoff. Feel free to use another legend system as long as there is justification.
  * Map directly using the GeoPandas library, or feel free to explore interactive maps in Plotly.

  Recall the principles of software engineering (consider writing a function for code that may be reused).

2. Show the relationship between log SCI and log distance in a figure. Highlight the points of at least one user geography (country or NUTS 2). Fit a curve to the data and plot it along with the scatterplot.

3. Visualize the distributions of log SCI by income bin (e.g. low, medium, high) of the user region. Comment.

Bonus: Explore the data and feel free to visualize whatever you would like. Make sure you understand the variables and compute summary statistics.

# Submission ⭐

Congratulations you finished Module 2! We will handle all exercise submissions on GitHub Classroom. Please push your changes to your team repository folder on GitHub Classroom.